In [ ]:
import model_backend as mdc
from pathlib import Path
import xarray as xr
import rioxarray as rioxr 
import numpy as np 

In [ ]:
def make_test_data(tmp_path) -> xr.Dataset:
    # Create proper geographical coordinates
    lon = np.linspace(-180, 180, 10)
    lat = np.linspace(-90, 90, 10)
    data = np.random.rand(10, 10)
    
    # Create dataset with proper dimension order for geospatial data
    ds = xr.Dataset(
        data_vars={
            "temp": (("y", "x"), data),
        },
        coords={
            "y": lat,
            "x": lon,
        },
    )
    
    # Save to netCDF
    ds.to_netcdf(tmp_path / "test_data.nc", mode="w", format="NETCDF4")
    
    
    # First load with netCDF engine
    with xr.open_dataset(tmp_path / "test_data.nc") as data:
    
        # Properly set spatial attributes needed by rioxarray - this is what makes .rio work
        data.rio.set_spatial_dims(x_dim="x", y_dim="y", inplace=True)
        data.rio.write_crs("EPSG:4326", inplace=True)
        ds = data.copy()
        
    return ds

In [ ]:
ds = make_test_data(Path("./"))
ds

In [ ]:

# Save to netCDF

# First load with netCDF engine
with xr.open_dataset("test_data.nc") as ds:

    print(ds.rio.crs)  # Check if CRS is set
    print(ds)
    # Properly set spatial attributes needed by rioxarray - this is what makes .rio work
    # ds.rio.set_spatial_dims(x_dim="x", y_dim="y", inplace=True)
    ds.rio.write_crs("EPSG:4326", inplace=True)
    ds

In [ ]:
ds

In [ ]:
ds.rio.crs

In [ ]:
ds.rio.write_crs(None, inplace=True)

# Build model instance

data is assumed to live in a directory: 

data/in

relative to the base directory of the package

In [ ]:
path = Path.cwd().parent / "data" / "in"
path

In [ ]:
model = mdc.JModel(
    input = path / "ERA5land_global_t2m_dailyStats_mean_01Deg_2024_08_data.nc", 
    output = Path.cwd().parent / 'data' / "out" / "output_map.nc", 
    r0_path= path / "R0_pip_stats.csv",
    run_mode="forbidden", 
    grid_data_baseurl="https://gisco-services.ec.europa.eu/distribution/v2/nuts",
    nuts_level =3, 
    resolution = "10M", 
    year=2024, 
)

# Run model -> load data -> apply model transformations -> save back to netcdf

In [ ]:
model.run()

# Have a look at produced data

In [ ]:
data = xr.open_dataset(
            Path.cwd().parent / 'data' / "out" / "output_map.nc", engine="rasterio"
        )

In [ ]:
data

In [ ]:
data["r0"].mean(dim="valid_time").plot(cmap="viridis")